In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/alexlei/Downloads/accepted_2007_to_2018Q4.csv', index_col=0, low_memory=False)

In [3]:
#pd.options.display.max_rows = 200
#df.dtypes

In [4]:
#df.head()

In [5]:
df.fillna('Null', inplace=True)

In [6]:
# Preparing for X and y

In [7]:
y = df['loan_status']

In [8]:
X = df.drop(df.loc[:, 'hardship_flag':'settlement_term'].columns, axis=1)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
# Dropping NA in y will cause inconsistency in rows between y and X
# y = y.dropna()

In [12]:
# Dropping NA in X will remove all rows
# X = X.dropna()

In [13]:
# Create an Explainable Boosting Machine (EBM)
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier()

# Fit the EBM on the training data
ebm.fit(X_train, y_train)

KeyboardInterrupt: 